In [ ]:
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from skimage.transform import resize
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
from yolo_utils import (
    read_classes,
    read_anchors,
    generate_colors,
    preprocess_image,
    draw_boxes,
    scale_boxes,
)
from yad2k.models.keras_yolo import (
    yolo_head,
    yolo_boxes_to_corners,
    preprocess_true_boxes,
    yolo_loss,
    yolo_body,
)

%matplotlib inline

In [ ]:
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold=0.6):
    """function for filtering the boxes based on their probabilities and threshold."""
    box_scores = box_confidence * box_class_probs
    box_classes = K.argmax(box_scores, -1)
    box_class_scores = K.max(box_scores, -1)
    filtering_mask = box_class_scores > threshold
    scores = tf.boolean_mask(box_class_scores, filtering_mask)
    boxes = tf.boolean_mask(boxes, filtering_mask)
    classes = tf.boolean_mask(box_classes, filtering_mask)

    return scores, boxes, classes

In [ ]:
def iou(box1, box2):
    """function for calculating the intersection over union of two bounding boxes."""
    xi1 = max(box1[0], box2[0])
    yi1 = max(box1[1], box2[1])
    xi2 = min(box1[2], box2[2])
    yi2 = min(box1[3], box2[3])
    inter_area = (yi2 - yi1) * (xi2 - xi1)
    box1_area = (box1[3] - box1[1]) * (box1[2] - box1[0])
    box2_area = (box2[3] - box2[1]) * (box2[2] - box2[0])
    union_area = box1_area + box2_area - inter_area
    iou = inter_area / union_area

    return iou

In [ ]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes=10, iou_threshold=0.5):
    max_boxes_tensor = K.variable(max_boxes, dtype="int32")
    K.get_session().run(tf.variables_initializer([max_boxes_tensor]))
    nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes, iou_threshold)
    scores = K.gather(scores, nms_indices)
    boxes = K.gather(boxes, nms_indices)
    classes = K.gather(classes, nms_indices)

    return scores, boxes, classes

In [ ]:
yolo_outputs = (
    tf.random_normal([19, 19, 5, 1], mean=1, stddev=4, seed=1),
    tf.random_normal([19, 19, 5, 2], mean=1, stddev=4, seed=1),
    tf.random_normal([19, 19, 5, 2], mean=1, stddev=4, seed=1),
    tf.random_normal([19, 19, 5, 80], mean=1, stddev=4, seed=1),
)

In [ ]:
def yolo_eval(yolo_outputs, image_shape = (720., 1280.), max_boxes=10, score_threshold=.6, iou_threshold=.5):
    box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs
    boxes = yolo_boxes_to_corners(box_xy, box_wh)
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = score_threshold)
    boxes = scale_boxes(boxes, image_shape)
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes, iou_threshold)

    return scores, boxes, classes